# Groundwater hydrology

This example comes from exercises 4.2.1-4.2.3 of the book *Inverse Problems in Groundwater Modeling* by Ne-Zhung Sun.
Here we are interested in the flow of groundwater in a saturated aquifer.
The conserved quantity is water mass, and the field we are solving for is the *hydraulic head* $h$.
The hydraulic head has units of length; if we were to drill into the aquifer, the hydraulic head is how far up the borehole the groundwater would rise to.
In a saturated porous medium, the hydraulic head is greater or equal to the thickness $H$ of the aquifer itself.
Later we'll look at unsaturated porous media.

The relation between the actual volume of water stored per unit volume of the porous medium is quantified in a coefficient called the *storativity* $S$.
Using the notation from class, our extensive density $G$ is water volume per unit footprint area, and the relation between the extensive density and the field to be solved for is
$$G = S\cdot h.$$
In some media, the storativity itself depends on the hydraulic head, but we'll assume it's a constant for now.

In this demo we're going to use a 2D setup in the $x$ and $y$ directions.
We'll use a set of equations for groundwater flow that assume some depth-averaging so that the vertical dependence is ignorable.

*Darcy's law* states that the flux of water through the aquifer is proportional to the gradient in hydraulic head:
$$F = -KH\nabla h$$
where $H$ is the aquifer thickness and $K$ is the *hydraulic conductivity* of the porous medium, which has units of length / time.
The hydraulic conductivity measures the ease with which water can percolate through the medium; sand and gravel have high conductivities, clays very low.

The sources $q$ of water come from infiltration from above, say from rainwater, while the sinks are loss of water to an aquifer below or direct pumping.

### Variational form

This is all worth writing out in a table so it's easy to refer back to.

| Name | Symbol | Units
| ---- | ------ | -----
| Hydraulic head | $h$ | length
| Water flux | $F$ | length${}^2$ / time
| Aquifer thickness | $H$| length
| Hydraulic conductivity | $K$ | length / time
| Storativity | $S$ | dimensionless
| Water sources | $Q$ | length / time
| External fluxes | $f$ | length${}^2$ / time

Putting everything together, the variational form of the problem is to find the hydraulic head $h$ for which
$$\int_\Omega\left(S\frac{\partial h}{\partial t}\phi + KH\nabla h\cdot\nabla\phi - Q\phi\right)dx - \int_{\Gamma_N} f\,d\gamma = 0$$
for all test functions $\phi$.
Here $\Gamma_N$ denotes the segment of the boundary where there are external fluxes.
On the remaining part $\Gamma_D$ of the domain boundary, we impose a fixed value of the hydraulic head:
$$h|_{\Gamma_D} = h_{\text{ext}}$$
Remember that fixed or proportional fluxes are *natural* boundary conditions (we incorporate them into the variational form) while Dirichlet conditions are *essential* (we have to modify the linear system and test functions).

### Geometry

First, we need to create a spatial domain.
Use a length $L_x$ of 6500m, a height $L_y$ of 4500m, and pick some sane-looking values for the number of points in the $x$ and $y$ directions.
You can pick `nx` and then set `ny = int(Ly / Lx * nx)` in order to make sure the triangles have the same aspect ratio.

In [ ]:
import matplotlib.pyplot as plt
import firedrake
from firedrake import Constant, conditional, inner, grad, dx, ds

Lx = ...
Ly = ...
nx = ...
ny = ...
mesh = firedrake.RectangleMesh(
    nx=nx,
    ny=ny,
    Lx=Lx,
    Ly=Ly,
    diagonal="crossed",
)

Remember when I said that the most common thing to mess up is boundary conditions?
The code below will plot the mesh and show the different boundary segments with different colors.
This is important because we're going to fix the hydraulic head on some of the boundary segments and fix the flux on others.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(mesh, boundary_kw={"linewidth": 5}, axes=ax)
ax.legend();

Next we're going to make a finite element.
This has three ingredients.
First and most important is the *family*.
Here, we're using "CG" for "Continuous Galerkin".
We'll see other families later in this class.
Next is the cell, which in this case is triangles -- this can also be `"interval"` in 1D, `"quadrilateral"` in 2D, or `"tetrahedron"` in 3D.
Finally, the *degree* specieis what polynomial degree we'll use in each element.
We'll start with 1, and you can try this with degree 2 later.

In [ ]:
element = firedrake.FiniteElement(family="CG", cell="triangle", degree=1)

Now we create a function space on our mesh with this choice of element.

In [ ]:
V = firedrake.FunctionSpace(mesh, element)

### Storativity and conductivity

Make the aquifer thickness a constant 50m and the storativity a constant $10^{-4}$.

In [ ]:
H = ...
S = ...

The aquifer is divided into three different zones, with boundaries at $x$ = 2500m and 4500m.
The conductivity is:
$$K = \begin{cases} 10 m/d & 0m \le x < 2500m \\ 20 m/d & 2500m \le x < 4500m \\ 40 m/d & 4500m \le x \le 6500m\end{cases}$$
Make an expression for the conductivity field below using `firedrake.conditional`.
You could nest one conditional inside another, or you could add up several conditionals.
If it's too confusing to make three zones at first, just make two to start.

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)
expr = conditional(
    ...,
    ...,
    ...,
)

Next we'll create the conductivity field and plot it.

In [ ]:
K = firedrake.Function(V)
K.interpolate(expr);

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
colors = firedrake.tripcolor(K, axes=ax)
fig.colorbar(colors, label="conductivity (m/day)");

You should see a field with discontinuities at the breakpoints we defined above.

### Sources

The problem from the book specifies pumping of 2000m${}^3$ / day at a point in the right-most zone.
We can put in a point sink but it takes some extra finagling, so instead we'll approximate it as a Gaussian.
Make the expression below

$$w = \exp\left(-\frac{(x - x_s)^2 + (y - y_s)^2}{r^2}\right)$$

where $r$ = 100m, $x_s$ = 5500m, $y_s$ = 2250m.
To get the right symbolic exponential function, you'll want to use `firedrake.exp` rather than, say, the numpy or sympy `exp` functions.

In [ ]:
w = ...

We want to get a sink field that adds up to the original pumping rate.
First, we'll evaluate the integral of $w$ over the whole domain by using the `assemble` function:

In [ ]:
w_total = firedrake.assemble(w * dx)  # <- the integral of `w`
print(f"{w_total:0.2f}")

Now create an expression for $Q$ as

$$Q = -2000\frac{\text{m}^3}{\text{day}} \cdot \frac{w}{\int_\Omega w\;dx}$$

In [ ]:
expr = ...

In order to make sure we did everything right, evaluate the integral of this expression using `assemble`.
You should get -2000.0.

In [ ]:
firedrake.assemble(expr * dx)

Now make a function $Q$ and interpolate this expression to it so we can plot it.
You should see something that looks roughly like a point sink.

In [ ]:
Q = ...

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
colors = firedrake.tripcolor(Q, axes=ax)
fig.colorbar(colors, label="m / day");

### Boundary conditions

Here we'll assume impermeable boundaries on the top, bottom, and right-hand sides, and a fixed value of 100m hydraulic head on the left-hand side.
Make the boundary condition below.
Scroll up to look at the plot of the mesh to find the right boundary ID.

In [ ]:
bc = ...

We don't have to do anything for the remaining boundaries since the fluxes there are zero, and fixed flux is a natural boundary condition.

### Modeling

Now make the initial hydraulic head field a constant 100m.

In [ ]:
h_0 = ...

We'll choose a final time of 1.5 days and a timestep of 1/24 days (i.e. 1 hour).

In [ ]:
final_time = 1.5
num_steps = int(final_time * 24)
timestep = final_time / num_steps
δt = Constant(timestep)

Next we'll create variables for the current and previous hydraulic head.

In [ ]:
h = firedrake.Function(V)
h.assign(h_0)
h_n = h.copy(deepcopy=True)

The variational form of the groundwater flow equation using backward timestepping is:
$$\int_\Omega\left(S\frac{h - h_n}{\delta t}\phi + KH\nabla h\cdot\nabla\phi - Q\phi\right)dx = 0$$
Make the variational form below.

In [ ]:
ϕ = ...
F = ...

Now make the timestepping loop to compute the hydraulic head at the final time.

Make a plot of the final value.
The hydraulic head should be a little lower in the vicinity of the pumping well.
If you messed up the sign of $Q$, you'll see that the hydraulic head is higher instead of lower.
How do I know that this is the kind of mistake one could make?
How indeed.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
colors = firedrake.tripcolor(h, axes=ax)
fig.colorbar(colors, label="m");

The code below shows how we can extract values of the function $h$ at some specific points by using `h.at`.

In [ ]:
import numpy as np
xs = np.linspace(0.0, Lx, 66)
ys = 2250.0 * np.ones_like(xs)
X = np.column_stack((xs, ys))
h_transect = h.at(X)

Here we can see from the transect plot that there's a change in the slope at the interfaces 2500m and 4500m between the parts of the aquifer with different conductivities.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("x (m)")
ax.set_ylabel("h (m)")
ax.axvline(2500.0, color="Grey", linestyle="dashed")
ax.axvline(4500.0, color="Grey", linestyle="dashed")
ax.plot(xs, h_transect);